
# Binary Prediction of Poisonous Mushrooms 
## Supervised Learning - IART 2nd Assignment

The second practical assignment of IART focuses on the application of Supervised Learning techniques in
the context of classification problems. The goal is to develop and evaluate machine learning models capable
of learning from labeled data to make accurate predictions regarding a specific target variable (or concept).

## Machine Learning Problem
In this notebook, we will work with a dataset containing information about various mushroom species to predict whether a mushroom is poisonous or edible, in a binary classification problem. Using this dataset, we will explore different classification algorithms and evaluate their performance in predicting the target variable.


### Methodology
Using the previously balanced dataset, we will apply various supervised learning algorithms to build a model that can accurately classify mushrooms as poisonous or edible. The dataset will be split into training and testing sets, and the models will be evaluated based on their performance metrics, including accuracy, precision, recall, and F1-score. We will use Decision Trees, K-Nearest Neighbors (KNN), and Support Vector Machines (SVM) as our classification algorithms. The models will be trained on the training set and evaluated on the testing set to assess their generalization performance. Additionally, we will perform hyperparameter tuning to optimize the models for better accuracy.

### 4. Load Libraries and Dataset

In [ ]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc

# Load the dataset
df = pd.read_csv('data/balanced_train.csv')

### 5. First Model - Decision Tree Classifier
Decision Trees are a popular and interpretable classification algorithm that can handle both categorical and numerical data. They work by recursively splitting the data based on feature values to create a tree-like structure, where each leaf node represents a class label.